In [1]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
import util

In [2]:
XFILE = 'data/imu_proc01-Feb-2018.csv'
YFILE = 'data/vicon_proc01-Feb-2018.csv'

In [3]:
def svm_model_sel(X_train, y_train):
    # kernels = ['rbf', 'linear', 'poly']
    Cs = [0.1, 1, 10, 100]
    gammas = [0.001, 0.01, 0.1, 1]
    epsilons = [0.01, 0.1, 1, 10]
    param_grid = {'C': Cs, 'epsilon': epsilons, 'gamma': gammas}
    model = SVR()
    param_search(X_train, y_train, model, param_grid)

In [4]:
def bayesian_model_sel(X_train, y_train):
    alpha1s = [1e-6, 1e-5, 1e-4, 0.001, 0.01, 0.1, 1, 10]
    alpha2s = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
    lambda1s = [1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6]
    lambda2s = [1e-6, 1e-5, 1e-4, 0.001, 0.01]
    param_grid = {'alpha_1': alpha1s, 'alpha_2': alpha2s,
                  'lambda_1': lambda1s, 'lambda_2': lambda2s}
    model = BayesianRidge()
    param_search(X_train, y_train, model, param_grid)

In [5]:
def mlp_model_sel(X_train, y_train):
    hidden_sizes = [10, 100, 500, 1000]
    activations = ['relu', 'tanh']
    alphas = [0.01, 0.1, 1, 10]
    # max_iters = [100, 500, 1500]
    param_grid = {'hidden_layer_sizes': hidden_sizes,
                  'alpha': alphas,
                  'activation': activations}
    model = MLPRegressor(max_iter=500)
    param_search(X_train, y_train, model, param_grid)

In [6]:
def param_search(X_train, y_train, model, param_grid):
    for yd in range(y_train.shape[1]):
        print('Searching params for y_' + str(yd) + '...')
        grid_search = GridSearchCV(model, param_grid, 'neg_mean_squared_error', n_jobs=-1, cv=10)
        grid_search.fit(X_train, y_train[:, yd])
        best_params = grid_search.best_params_
        print(best_params)
        print('best score:', grid_search.best_score_)

In [7]:
X = np.genfromtxt(XFILE, delimiter=',')
X = X[:, 10:]  # wrist only
y = np.genfromtxt(YFILE, delimiter=',')

In [8]:
X_train, X_test, y_train, y_test = util.test_split(X, y, 1000, 0.1)
scaler = StandardScaler().fit(X_train)
# scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print('===SVM===')
svm_model_sel(X_train, y_train)